# Extended Large Language Model (LLM) with Transformer Blocks

This notebook builds on the mini LLM example by incorporating self-attention and transformer blocks to simulate a more realistic architecture.
We implement a simplified version of GPT-style decoder blocks.

# Step 1: Import Required Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# Step 2: Load and Preprocess Text

In [ ]:
text = open("RomeoJuliet.txt", encoding='utf-8').read()
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# Step 3: Hyperparameters and Batch Loader

In [ ]:
block_size = 128
batch_size = 64
max_iters = 2000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)

In [ ]:
# Step 4: Transformer Block with Self-Attention

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, n_embed, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.dropout = nn.Dropout(0.1)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        return wei @ v

class TransformerBlock(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        head_size = n_embed
        self.sa = SelfAttention(n_embed, head_size)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ffwd = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
        )
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class TransformerLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embed = nn.Embedding(vocab_size, n_embed)
        self.pos_embed = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(
            TransformerBlock(n_embed),
            TransformerBlock(n_embed),
        )
        self.ln_f = nn.LayerNorm(n_embed)
        self.head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx):
        B, T = idx.shape
        tok_emb = self.token_embed(idx)
        pos_emb = self.pos_embed(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.head(x)
        return logits

n_embed = 128
model = TransformerLanguageModel().to(device)

In [ ]:
# Step 5: Train the Model

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
losses = []

for step in tqdm(range(max_iters)):
    xb, yb = get_batch('train')
    logits = model(xb)
    B, T, C = logits.shape
    loss = F.cross_entropy(logits.view(B * T, C), yb.view(B * T))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if step % eval_interval == 0:
        print(f"Step {step}: loss = {loss.item():.4f}")
        losses.append(loss.item())

plt.plot(losses)
plt.title("Training Loss")
plt.xlabel("Iterations (x100)")
plt.ylabel("Loss")
plt.show()

In [ ]:
# Step 6: Generate Text

In [ ]:
def generate(model, start_text, max_new_tokens=200):
    model.eval()
    idx = torch.tensor([encode(start_text)], dtype=torch.long).to(device)
    for _ in range(max_new_tokens):
        idx_crop = idx[:, -block_size:]
        logits = model(idx_crop)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, next_token), dim=1)
    return decode(idx[0].tolist())

print(generate(model, "ROMEO.\n"))